In [ ]:
###import jetbot###
import os
import time
import threading
from jetbot import Robot
import numpy as np
import math
import cv2
import PIL.Image
from IPython.display import display
import ipywidgets
import traitlets

robot = Robot()

In [ ]:
###import yolov4###
import pycuda.autoinit  # This is needed for initializing CUDA driver
from utils.yolo_classes1 import get_cls_dict
from utils.camera import add_camera_args, Camera
from utils.display import open_window, set_display, show_fps
from utils.visualization import BBoxVisualization
from utils.yolo_with_plugins import get_input_shape, TrtYOLO
from operator import itemgetter

cls_dict = get_cls_dict(14)
vis = BBoxVisualization(cls_dict)
trt_yolo = TrtYOLO('yolov4-bdd', (288, 288), 14, False)
conf_th = 0.4

In [ ]:
###import camera###
def gstreamer_pipeline(
    sensor_id=0,
    sensor_mode=3,
    capture_width=600,
    capture_height=450,
    display_width=640,
    display_height=360,
    framerate=25,
    flip_method=0,
):
    return (
        "nvarguscamerasrc sensor-id=%d sensor-mode=%d ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            sensor_id,
            sensor_mode,
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

cam = cv2.VideoCapture(gstreamer_pipeline(flip_method=2,sensor_id = 1), cv2.CAP_GSTREAMER)
cam1 = cv2.VideoCapture(gstreamer_pipeline(flip_method=4,sensor_id = 0), cv2.CAP_GSTREAMER)

if not cam.isOpened():
    raise SystemExit('ERROR: failed to open camera!')
if not cam1.isOpened():
    raise SystemExit('ERROR: failed to open camera!')

w = 640
h = 360
roiHeightRatio = 8/12

live = ipywidgets.Image()

In [ ]:
height = 8.8
alpha = 84*math.pi/180
theta = math.atan(14/height)
f = h/( 2*math.tan(alpha-theta) )

def distance(bbox_y):
    d = (h/2) - bbox_y
    delta = math.atan(d/f)
    dist = height*math.tan(alpha+delta)
    if( dist < 0 ):
        return 999
    return dist

In [ ]:
def roi(img, vertices):
    mask = np.zeros_like(img)

    if len(img.shape) >2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,255,255)
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)

    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image   


def draw_area(img, left, right):
    vertices = np.array([[(left[0],left[1]),(left[2],left[3]),(right[2],right[3]),(right[0],right[1])]],dtype=np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly( mask,vertices, (216,168,74))
    masked_img = cv2.addWeighted(img, 0.7, mask, 0.4,0)

    return masked_img, mask


def FitLine(line_array, line_degree, constant):
    [line_x,line_y,line_x1,line_y1] = [int((h-constant)/line_degree),h,int(((h*roiHeightRatio)-constant)/line_degree),h*roiHeightRatio]

    return [line_x, line_y, line_x1, line_y1]


def LineFunc(x,y,x1,y1):
    if (x1-x) == 0:
        line_degree = 999.0
    else:
        line_degree = (y1-y)/(x1-x)

    constant = y-line_degree*x

    return line_degree, constant


def LineFuncLen(line_degree, constant):
    if (line_degree == 0):
        length = w
    elif ( line_degree == 999.0 ):
        length = h-h*roiHeightRatio
    else:
        coordinate = []
        #위, 아래, 왼쪽, 오른쪽
        x ,y  = -1*constant/line_degree, 0
        x1,y1 = ((h-h*roiHeightRatio) - constant)/line_degree, (h-h*roiHeightRatio)
        x2,y2 = 0, constant
        x3,y3 = w, line_degree*w + constant

        if ( 0 <= x <= w ):
            coordinate.append(x)
            coordinate.append(y)
        if ( 0 <= x1 <= w ):
            coordinate.append(x1)
            coordinate.append(y1)
        if ( 0 < y2 < h-h*roiHeightRatio):
            coordinate.append(x2)
            coordinate.append(y2)
        if ( 0 < y3 < h-h*roiHeightRatio):
            coordinate.append(x3)
            coordinate.append(y3)
        
        if( len(coordinate) < 4 ):
            coordinate.append(x1)
            coordinate.append(y1)
            
        length = ((coordinate[0]-coordinate[2])**2+(coordinate[1]-coordinate[3])**2)**0.5
        
    return length


def FindLRLines_new_turn(img_color, vertices, previous_left, previous_right, LRisLine, leftCount, rightCount):
    leftLineFound = False
    rightLineFound = False
    
    img_cropped = img_color[int(h*roiHeightRatio):h, 0:w]
    #img_cropped = img_color
    img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
    ret, img_binary = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    #검출된 사각형이 없는 경우의 차선
    left_fit_line = previous_left[:4]
    right_fit_line = previous_right[:4]

    contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #for cnt in contours[::-1]:
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if ( 200 < area < 7000 ):
            cnt = np.squeeze(cnt)
            
            cnt_transpose = np.transpose(cnt)
            max_y_index = np.argmax(cnt_transpose[1])
            min_y_index = np.argmin(cnt_transpose[1])

            max_y = cnt_transpose[1][max_y_index]
            min_y = cnt_transpose[1][min_y_index]
            
            if( max_y > 100 and max_y-min_y < 50 ):
                continue

            mid_y1 = round(min_y + (max_y-min_y)*1/8)
            mid_y2 = round(min_y + (max_y-min_y)*3/8)
           
            mid_y1_index = np.where(cnt_transpose[1] == mid_y1)              
            mid_y2_index = np.where(cnt_transpose[1] == mid_y2)
  
            x = int((cnt[mid_y1_index[0][0]][0] + cnt[mid_y1_index[0][len(mid_y1_index[0])-1]][0])/2)
            y = mid_y1 
            x1 = int((cnt[mid_y2_index[0][0]][0] + cnt[mid_y2_index[0][len(mid_y2_index[0])-1]][0])/2)
            y1 = mid_y2
            line_degree, constant = LineFunc(x,y,x1,y1)
            
            # 차선의 경우 폭이 일정치 이하이기 때문에 그 이상의 값들은 차선이 아닌 것으로 간주하여 필터링 진행
            if( cnt[mid_y1_index[0][len(mid_y1_index[0])-1]][0] - cnt[mid_y1_index[0][0]][0] > 40 ):
                continue
            
            temp = [x,y + h*roiHeightRatio, x1, y1 + h*roiHeightRatio]
            temp = FitLine(temp, line_degree, constant+h*roiHeightRatio)
            
            
            if( (previous_left[0] - 100 <= temp[0] <= previous_left[0] + 100) and leftLineFound==False ):
                left_fit_line = temp[:4]
                leftLineFound = True
                
                x = int((max_y-constant)/line_degree)
                y = max_y
                x1 = int((min_y-constant)/line_degree)
                y1 = min_y

                contour_length = ((x-x1)**2+(y-y1)**2)**0.5
                LineFunc_length = LineFuncLen(line_degree, constant)
                LineError = contour_length/LineFunc_length
            
                if(LineError >= 0.9):
                    LRisLine[0] = 1
                else:
                    LRisLine[0] = 0
    

            if( (previous_right[0] - 100 <= temp[0] <= previous_right[0] + 100)  and rightLineFound==False ):
                right_fit_line = temp[:4]
                rightLineFound = True
                
                x = int((max_y-constant)/line_degree)
                y = max_y
                x1 = int((min_y-constant)/line_degree)
                y1 = min_y

                contour_length = ((x-x1)**2+(y-y1)**2)**0.5
                LineFunc_length = LineFuncLen(line_degree, constant)
                LineError = contour_length/LineFunc_length
            
                if(LineError >= 0.9):
                    LRisLine[1] = 1
                else:
                    LRisLine[1] = 0
            
            if( leftLineFound and rightLineFound ):
                if( left_fit_line == right_fit_line):
                    left_fit_line = [0,h,0,h*roiHeightRatio]
                leftCount=0
                rightCount=0
                return left_fit_line, right_fit_line, LRisLine, leftCount, rightCount



    # 차선인식이 안된 경우, 왼쪽 차선은 왼쪽 끝 벽을, 오른쪽 차선은 오른쪽 끝 벽을 넣는다.  
    
    if( rightLineFound ): 
        leftCount+=1
    else:
        rightCount+=1
    
    if( leftCount >= 15 ):
        leftCount = 0
        left_fit_line = [0,h,0,h*roiHeightRatio]
    if( rightCount >= 15 ):
        rightCount = 0
        right_fit_line = [w,h,w,h*roiHeightRatio]
    return left_fit_line, right_fit_line, LRisLine, leftCount, rightCount

In [ ]:
def Calculate_steering(left, right, w, speed, keep):
    mid_x = w/2
    a = w/2 - left[0]
    b = right[0] - w/2
    
    x1,y1,x2,y2 = left
    x3,y3,x4,y4 = right
    l, _ = LineFunc(x1,y1,x2,y2)
    r, _ = LineFunc(x3,y3,x4,y4)
    #print(r)
    
    if(keep==3):
        steering = -0.027
        #steering = -0.017
            
    elif(a>0 and b>0):
        
        if( (a < w*6/16) and (l > -0.75) ):
            steering = 0.06    # (+)우회전
            
        elif ( (b < w*6/16) and (r < 0.75) ):
            steering = -0.06    # (-)좌회전
            
        else:
            ratio = abs(a/b) 
            
            if( speed <= 0.15 ):
                if( 0.95 <= ratio <= 100/95 ):
                    steering = 0
                elif ( ratio < 0.95 ):
                    steering = 0.003
                else:
                    steering = -0.003
            elif( speed <= 0.25 ):
                if( 0.95 <= ratio <= 100/95 ):
                    steering = 0
                elif ( ratio < 0.95 ):
                    steering = 0.007
                else:
                    steering = -0.007
            else:
                if( 0.95 <= ratio <= 100/95 ):
                    steering = 0
                elif ( ratio < 0.95 ):
                    steering = 0.012
                else:
                    steering = -0.012
    
    else:
        steering = 0
    #print(speed)
    return steering

In [ ]:
global img_copy
global img_color
global img_back
global boxes, confs, clss
global turn_left
global turn_right
global steering
global boxList
#global fps
steering = 0
speed = 0
keep = 1    # 1:중앙유지중   2:차선변경 1단계  3:차선변경 2단계
turn = 0    # 1:왼쪽차선변경 2:오른쪽차선변경
turn_LR = 0 # 1:왼쪽차선변경 2:오른쪽차선변경

LR = [1,1]
LR_back = [1,1]

left_slope = 0
right_slope = 0
previous_left = [0,0,0,0]
previous_right = [600,0,600,0]

previous_left_back = [0,0,0,0]
previous_right_back = [590,0,590,0]
Pos = [0,0,0,0,0,0]

fps = 0.0


In [ ]:
def threadMain():
    global keep
    global turn
    global previous_left, previous_right
    global speed   
    global previous_left_back, previous_right_back
    global img_back
    global steering
    global img_color
    global img_back
    global boxes, confs, clss, boxes1, confs1, clss1
    global turn_left
    global turn_right
    global Pos
    global LR, LR_back   
    cnt = 0
    car_cnt = 0
    global angle
    global angle_last
    global error_sum
    global steering
    global img_final_pre
    time.sleep(5) #yolo프레임 회복 기다리기
    flag_choowall = 0
    flag_return = False
    speed = 0 ###
    global fps
    
    
    tic = time.time()
    leftCount=0 
    rightCount=0 
    leftBackCount=0 
    rightBackCount=0

    while True:
        ret, img_color = cam.read()
        ret1, img_back = cam1.read()
        img_final_pre = np.vstack((img_color, img_back))
        previous_left, previous_right, LR, leftCount, rightCount = FindLRLines_new_turn(img_color, vertices, previous_left, previous_right, LR, leftCount, rightCount)
        if( flag_return and steering==0): #차선 변경시 뒤차선의 left right도 바꿔줘야됨
            previous_right_back = [w,h,w,h*roiHeightRatio]
            previous_left_back = [0,h,0,h*roiHeightRatio]
            flag_return = False
        previous_left_back, previous_right_back, LR_back, leftBackCount, rightBackCount = FindLRLines_new_turn(img_back, vertices, previous_left_back, previous_right_back, LR_back, leftBackCount, rightBackCount)
        if(keep == 1):###중앙유지/차선인식->조향제어###
            if(len(boxes)>0):
                flag = True #차량이 있지만, 중앙에 없는 경우를 위한 flag
                for bb, cf, cl in zip(boxes, confs, clss):
                    if(cl==2):
                        BbM_x, BbM_y = (bb[0] + bb[2]) // 2, bb[3] #x_min, y_min, x_max, y_max = bb[0], bb[1], bb[2], bb[3]
                        if( BbM_y > 360 ):
                            continue
                        dist = distance(BbM_y) 
                        x_rb, y_rb, x_ru, y_ru = previous_right
                        x_lb, y_lb, x_lu, y_lu = previous_left
                        if (BbM_y - y_lb) - ((y_lu-y_lb)/(x_lu-x_lb+0.001))*(BbM_x-x_lb) > 0 and (BbM_y - y_rb) - ((y_ru-y_rb)/(x_ru-x_rb+0.001))*(BbM_x-x_rb) > 0:

                            flag = False
                            if ( dist <= 35 ):
                                speed = 0
                                steering = 0
                            else:
                                speed = min( 0.3, 3e-5*(dist*dist) - 0.0014*dist + 0.1674 )
                                
                                steering = Calculate_steering(previous_left, previous_right, w, speed, keep) #steering을 구할 때 speed도 고려해야 하므로 인자로 speed도 넣어줘야 할 것
                                if( 35 < dist < 50 ):
                                    keep = 2
                                    Pos[1] = 1
                            break
                if(flag):
                    speed += 0.01
                    speed = min(0.3, speed)
                    steering = Calculate_steering(previous_left, previous_right, w, speed, keep) #steering을 구할 때 speed도 고려해야 하므로 인자로 speed도 넣어줘야 할 것

            else:
                speed = min(0.3, speed + 0.01)
                steering = Calculate_steering(previous_left, previous_right, w, speed, keep)
                
               
            robot.left_motor.value = max(min(speed + steering + 0.023, 1.0), 0.0) # 0.008은 100
            robot.right_motor.value = max(min(speed - steering, 1.0), 0.0)

                                
        elif(keep == 2):###상황판단###
            cnt += 1
            
            x_rb, y_rb, x_ru, y_ru = previous_right
            x_lb, y_lb, x_lu, y_lu = previous_left
            x_rb_back, y_rb_back, x_ru_back, y_ru_back = previous_right_back[0],previous_right_back[1]+360,previous_right_back[2],previous_right_back[3]+360 
            x_lb_back, y_lb_back, x_lu_back, y_lu_back = previous_left_back[0],previous_left_back[1]+360,previous_left_back[2],previous_left_back[3]+360
            
            if(len(boxes)>0):
                for bb, cf, cl in zip(boxes, confs, clss):
                    x_min, y_min, x_max, y_max = bb[0], bb[1], bb[2], bb[3]
                    BbM_y_c = bb[3]
                    BbM_x_c = (x_min + x_max) // 2
                    
                    if (cl == 2 and BbM_y_c <= 360):
                        if (BbM_y_c - y_lb) - ((y_lu-y_lb)/(x_lu-x_lb+0.001))*(BbM_x_c-x_lb) > 0 and (BbM_y_c - y_rb) - ((y_ru-y_rb)/(x_ru-x_rb+0.001))*(BbM_x_c-x_rb) > 0:
                            speed = min( 0.3, 3e-5*(dist*dist) - 0.0014*dist + 0.1674 )
                            steering = Calculate_steering(previous_left, previous_right, w, speed, keep)
                            Pos[1] = 1 #Front Mid
                        elif (BbM_y_c - y_lb) - ((y_lu-y_lb)/(x_lu-x_lb+0.001))*(BbM_x_c-x_lb) < 0:
                            Pos[0] = 1 #Front Left
                        else:
                            Pos[2] = 1 #Front Right
                    elif( cl==2 and BbM_y_c >360 ):
                        distBack = distance( BbM_y_c-360 )
                        if (BbM_y_c - y_lb_back) - ((y_lu_back-y_lb_back)/(x_lu_back-x_lb_back+0.001))*(BbM_x_c-x_lb_back) > 0 and (BbM_y_c - y_rb_back) - ((y_ru_back-y_rb_back)/(x_ru_back-x_rb_back+0.001))*(BbM_x_c-x_rb_back) > 0:
                            Pos[4] = 1 #back Mid
                        elif (BbM_y_c - y_lb_back) - ((y_lu_back-y_lb_back)/(x_lu_back-x_lb_back+0.001))*(BbM_x_c-x_lb_back) < 0 and distBack <= 30 :
                            Pos[3] = 1 #back Left
                        elif (BbM_y_c - y_rb_back) - ((y_ru_back-y_rb_back)/(x_ru_back-x_rb_back+0.001))*(BbM_x_c-x_rb_back) > 0 and distBack <= 30 :
                            Pos[5] = 1 #back Right

                       
                            
            robot.left_motor.value = max(min(speed + steering + 0.023, 1.0), 0.0) # 0.008은 100
            robot.right_motor.value = max(min(speed - steering, 1.0), 0.0)
            
            if(cnt == 8):        
                print(Pos)
                if (Pos[1]==1 and Pos[0]==0 and Pos[3]==0 and LR[0]==0): #왼쪽 차선에 앞차와 뒤차 없고 점선이면 turn left
                    print("turn_left")
                    keep = 3
                    turn = 1
                    speed = 0.2
                    previous_right = previous_left
                    previous_left = [-200,h,-200,int(h*roiHeightRatio)]
                    
                    
                elif(Pos[1]==1 and Pos[2]==0 and Pos[5]==0 and LR[1]==0):#오른쪽에 차선에 앞차와 뒤차 없고 점선이면 turn right
                    print("turn_right 불가")
                    keep = 1
                    Pos = [0,0,0,0,0,0]
                else:
                    print("잠시 대기")
                    keep = 1
                    Pos = [0,0,0,0,0,0]
                    
                cnt = 0
                    
        elif(keep == 3):###차선변경###
            if turn == 1:
                steering = Calculate_steering(previous_left, previous_right, w, speed, keep) #steering을 구할 때 speed도 고려해야 하므로 인자로 speed도 넣어줘야 할 것

                if( previous_left[0] > w*1/16 ):
                    flag_return = True
                    turn = 0
                    
                robot.left_motor.value = max(min(speed + steering + 0.023, 1.0), 0.0) # 0.008은 bias
                robot.right_motor.value = max(min(speed - steering, 1.0), 0.0)
                
            else:
                keep = 1
                
        
        toc = time.time()
        curr_fps = 1.0 / (toc - tic)
        # calculate an exponentially decaying average of fps number
        fps = curr_fps if fps == 0.0 else (fps*0.95 + curr_fps*0.05)
        tic = toc
        
        x, y, x1, y1 = int(previous_left[0]), int(previous_left[1]), int(previous_left[2]), int(previous_left[3])
        x2, y2, x3, y3 = int(previous_right[0]), int(previous_right[1]), int(previous_right[2]), int(previous_right[3])
        x4, y4, x5, y5 = int(previous_left_back[0]), int(previous_left_back[1] + 360), int(previous_left_back[2]), int(previous_left_back[3] + 360)
        x6, y6, x7, y7 = int(previous_right_back[0]), int(previous_right_back[1] + 360), int(previous_right_back[2]), int(previous_right_back[3] + 360)
        
        img_final = img_final_pre.copy()
        
        img_final = vis.draw_bboxes(img_final, boxes, confs, clss)

        if( LR[0] == 1 ):
            img_final = cv2.line(img_final, (x,y),(x1,y1),(0,0,255),3)
        else:
            img_final = cv2.line(img_final, (x,y),(x1,y1),(255,0,0),3)

        if( LR[1] == 1):
            img_final = cv2.line(img_final, (x2,y2),(x3,y3),(0,0,255),3)
        else:
            img_final = cv2.line(img_final, (x2,y2),(x3,y3),(255,0,0),3)

        if( LR_back[0] == 1):
            img_final = cv2.line(img_final, (x4,y4),(x5,y5),(0,0,255),3)
        else:
            img_final = cv2.line(img_final, (x4,y4),(x5,y5),(255,0,0),3)

        if( LR_back[1] == 1):
            img_final = cv2.line(img_final, (x6,y6),(x7,y7),(0,0,255),3)
        else:
            img_final = cv2.line(img_final, (x6,y6),(x7,y7),(255,0,0),3)        
        
        live.value = bytes(cv2.imencode('.jpg', img_final)[1])
                


t_main = threading.Thread(target=threadMain)


In [ ]:
ret, img_color = cam.read()  
ret1, img_back = cam1.read()
leftCount=0 
rightCount=0 
leftBackCount=0 
rightBackCount=0
print(ret, ret1)
vertices = np.array([[(0,int(h)),(0,int(h*roiHeightRatio)),(w,int(h*roiHeightRatio)),(w,int(h))]],dtype=np.int32)  # 이전 프레임 차선으로 roi 설정
boxes, confs, clss = trt_yolo.detect(img_color, conf_th)
boxes1, confs1, clss1 = trt_yolo.detect(img_back, conf_th)
keep=1
previous_left, previous_right, LR,leftCount, rightCount= FindLRLines_new_turn(img_color, vertices, previous_left, previous_right, LR,leftCount, rightCount)
previous_left_back, previous_right_back, LR_back, leftBackCount, rightBackCount = FindLRLines_new_turn(img_back, vertices, previous_left_back, previous_right_back, LR_back, leftBackCount, rightBackCount)
print(previous_left, previous_right, previous_left_back, previous_right_back)
img_final_pre = np.vstack((img_color, img_back))
steering = Calculate_steering(previous_left, previous_right, w, speed, keep)

live.value = bytes(cv2.imencode('.jpg', img_final_pre)[1])

In [ ]:
t_main.start()

while True:
    boxes, confs, clss = trt_yolo.detect(img_final_pre, conf_th)


In [ ]:
display(live)